## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**: `def daily_active_users(activity: pd.DataFrame) -> pd.DataFrame`
* I/O 禁止、不要な `print` や `sort_values` は使用しない
* Activity は重複行を含む可能性があるので、**(activity_date, user_id) でユニーク化してからカウント**

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  `Activity` データフレームから、**2019-07-27 を含む直近 30 日間**における、日別アクティブユーザー数を求める。
  あるユーザーが「その日に 1 回以上アクティビティを行っていれば」その日はアクティブとみなす。
  アクティビティ種別（`activity_type`）は `'open_session', 'end_session', 'scroll_down', 'send_message'` のいずれも有効とみなす。
  アクティブユーザー数が 0 の日は結果に含めない。

* `{{INPUT_DATAFRAMES}}`

  ```text
  activity: pd.DataFrame
  columns:
    - user_id       (int)
    - session_id    (int)
    - activity_date (date または datetime64[ns] 相当)
    - activity_type (category/str: 'open_session', 'end_session', 'scroll_down', 'send_message')
  ```

* `{{OUTPUT_COLUMNS_AND_RULES}}`

  ```text
  出力: pd.DataFrame
  columns:
    - day           (date)  … activity_date
    - active_users  (int)   … その日に 1 回以上アクティビティを行ったユニーク user_id 数

  ルール:
    - 対象期間: 2019-06-28 ～ 2019-07-27（両端含む）…「2019-07-27 を含む 30 日間」
    - アクティブ判定は「その日の (user_id, activity_date) が 1 回以上存在」
    - ユーザー数 0 の日は含めない（=そもそも行が存在しない）
    - 行順は任意（sort は不要）
  ```

---

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 期間フィルタ → `(user_id, activity_date)` でユニーク化 → `groupby` で日別カウント。

```python
import pandas as pd

def daily_active_users(activity: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['day', 'active_users']
    """
    # 期間フィルタ: 2019-07-27 を含む直近 30 日間 → 2019-06-28〜2019-07-27
    start = pd.to_datetime("2019-06-28")
    end = pd.to_datetime("2019-07-27")
    mask = (activity["activity_date"] >= start) & (activity["activity_date"] <= end)

    # 列最小化 + ユニーク化: 1 ユーザーが 1 日に複数行あっても 1 回とカウントする
    uniq = (
        activity.loc[mask, ["user_id", "activity_date"]]
        .drop_duplicates()
    )

    # 日別にユーザー数をカウント（ユーザー ID のユニーク数）
    result = (
        uniq
        .groupby("activity_date", as_index=False)["user_id"]
        .nunique()
    )

    # 列名を仕様どおりに整形
    result = result.rename(
        columns={
            "activity_date": "day",
            "user_id": "active_users",
        }
    )

    return result

Analyze Complexity
Runtime 316 ms
Beats 50.66%
Memory 68.35 MB
Beats 38.32%

```

---

## 3) アルゴリズム説明

使用している主な pandas API:

1. **期間フィルタ**:

   ```python
   mask = (activity["activity_date"] >= start) & (activity["activity_date"] <= end)
   activity.loc[mask, ...]
   ```

   * `activity_date` が `datetime64[ns]` / `date` でも比較可能。
   * 2019-06-28〜2019-07-27 に含まれる行だけを対象とする。

2. **列最小化 & ユニーク化**:

   ```python
   activity.loc[mask, ["user_id", "activity_date"]].drop_duplicates()
   ```

   * 使うのはアクティブ判定のキーだけなので、列を `user_id, activity_date` に絞る。
   * 同じユーザーが同じ日に複数アクティビティをしていても、
     `(user_id, activity_date)` のペアが一意になるように `drop_duplicates()`。

3. **日別アクティブユーザー数**:

   ```python
   uniq.groupby("activity_date", as_index=False)["user_id"].nunique()
   ```

   * `groupby("activity_date")` で日ごとにまとめる。
   * 各日について `user_id` のユニーク数（`nunique()`）を取り、アクティブユーザー数とする。
   * アクティビティが 1 件もない日付は `uniq` 自体に現れないため、
     自然と結果にも含まれない（=「0 の日は除外」要件を満たす）。

4. **列名整形**:

   ```python
   result.rename(columns={"activity_date": "day", "user_id": "active_users"})
   ```

   * 問題の要求どおり出力列名を `day`, `active_users` に揃える。
   * 並び順も `[day, active_users]` になるようにしている。

### NULL / 重複 / 型

* **重複**:
  `(user_id, activity_date)` で `drop_duplicates()` しているため、
  同じユーザーが同日に 10 回アクションしても **1 ユーザーとしてカウント**される。
* **NULL**:

  * `activity_date` や `user_id` が `NaN` の行は、条件に合致しても
    ペアとして扱いづらいため、通常はデータ仕様上「存在しない前提」。
    （もし存在すれば、`NaN` を含む行も 1 ユニットとして扱われるので、
    必要に応じて `dropna(subset=["user_id", "activity_date"])` を噛ませてもよい。）
* **型**:

  * `activity_date` が文字列の場合でも、`to_datetime` 済みなら問題なく比較できる。
  * `user_id` は `int` 前提だが、`nunique()` は dtype に依存せず機能する。

---

## 4) 計算量（概算）

Activity の行数を **N**、期間内に残る行数を **N₃₀** とすると:

1. **期間フィルタ** `activity.loc[mask, ...]`

   * 各行を 1 回見て条件を評価 → **O(N)**

2. **`drop_duplicates()`** on `["user_id", "activity_date"]`

   * ハッシュベースの重複除去 → **O(N₃₀)** 〜 **O(N₃₀ log N₃₀)** 近辺

3. **`groupby("activity_date")["user_id"].nunique()`**

   * グループ数を D（日数 ≤ 30）とすると
     **O(N₃₀)**（ハッシュ集約）程度

総じて、支配的なのは **O(N₃₀)**～**O(N₃₀ log N₃₀)**。
日数 D は最大 30 と小さいため、D によるオーバーヘッドは無視できるレベルです。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[activity DF] --> B[期間フィルタ 2019-06-28〜2019-07-27]
  B --> C[列最小化 user_id, activity_date]
  C --> D["(user_id, activity_date) でdrop_duplicates"]
  D --> E["groupby(activity_date) user_id.nunique()"]
  E --> F[列名を day, active_users に整形]
  F --> G["出力 DF (day, active_users)"]
```

この実装は

* I/O なし
* `print` / `sort_values` 不使用
* 指定シグネチャ・出力列仕様を満たす

ので、LeetCode などの Pandas 問題でもそのまま利用できます。

結論から言うと、

* **316ms / 上位 50% は十分合格ライン**
* LeetCode の Pandas 環境だと、**劇的な改善はほぼ出ません**
* とはいえ、**drop_duplicates を消す & 中間 DF を減らす**くらいのチューニング余地はあります

という感じです。

---

## 1) ロジック的な改善ポイント

今のコードはざっくりこんな流れでした：

```python
mask = (activity["activity_date"] >= start) & (activity["activity_date"] <= end)

uniq = (
    activity.loc[mask, ["user_id", "activity_date"]]
    .drop_duplicates()
)

result = (
    uniq
    .groupby("activity_date", as_index=False)["user_id"]
    .nunique()
)
```

ここで

* `(user_id, activity_date)` を `drop_duplicates` した後、
* さらに `groupby().nunique()` で `user_id` のユニーク数を数えている

ので、**「重複除去」を 2 回やっている**イメージになっています。

この問題は「日ごとの user_id のユニーク数」が欲しいだけなので、

* **`drop_duplicates` をやめて**
* そのまま `groupby().nunique()` だけにする

方がシンプル＆速くなる可能性が高いです。

---

## 2) 改訂版コード（中間処理を削る）

```python
import pandas as pd

def daily_active_users(activity: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['day', 'active_users']
    """
    # 定数は一度だけ to_datetime（ここはほぼオーバーヘッドになりません）
    start = pd.to_datetime("2019-06-28")
    end = pd.to_datetime("2019-07-27")

    # 期間フィルタ ＋ 列最小化
    filtered = activity.loc[
        (activity["activity_date"] >= start) & (activity["activity_date"] <= end),
        ["user_id", "activity_date"],
    ]

    # 日別に user_id のユニーク数をカウント
    result = (
        filtered
        .groupby("activity_date", as_index=False)["user_id"]
        .nunique()
        .rename(columns={"activity_date": "day", "user_id": "active_users"})
    )

    return result

Analyze Complexity
Runtime 341 ms
Beats 24.18%
Memory 68.30 MB
Beats 49.34%

```

### 変更点まとめ

1. **`drop_duplicates()` を削除**

   * `groupby("activity_date")["user_id"].nunique()` が「日 × user_id のユニーク数」を出してくれるので、
     事前の `(user_id, activity_date)` ユニーク化は不要です。
2. **マスクを直接 `.loc` の中で使っても良い**

   * メモリ的にはほぼ同じですが、コードが少しスリムになります。
3. **中間 DF を 1 つだけに削減**

   * `uniq` をなくして `filtered` → `result` の 2 段構成に。

これで計算量もメモリ使用量も、**わずかにだけ軽く**なります。

---

## 3) メモリ面の改善余地

LeetCode のメモリ 68MB / Beat 38% も、正直かなり十分ですが、

「理論上できること」としては：

1. **使わない列を早めに捨てる**

   ```python
   filtered = activity.loc[mask, ["user_id", "activity_date"]]
   ```

   → これはすでにやっています。`session_id` / `activity_type` を持ち回っていないので OK。

2. **型のダウンサイジング**

   * 実務なら、`user_id`, `session_id` を `int32` に落としたり、`activity_type` を `category` にしてメモリ削減が効きます。
   * ただし LeetCode では

     * 変換コスト > メリット
     * 評価環境の測り方次第
       となることが多く、あまりスコアに出ないことも多いです。

---

## 4) どこまで頑張るべきか

* Pandas 版で **Runtime Beats ~50%** は、
  「アルゴリズムも実装も特に悪くない」ラインです。
* Leaderboard 上位は

  * CPython のバージョン差
  * ジャストタイミングで速いマシンに当たる
  * 評価のランダム誤差
    なども効いてくるので、**数十 ms の差はほぼノイズ**です。

なので、

* ロジックをシンプルにして二重の重複除去をやめる（上記修正版）
* それ以上は「趣味の世界」

くらいに考えておくのが現実的かな、と思います。

「pandas は入出力のフレームだけ、ロジックは全部 NumPy」でゴリ押ししてみましょう。

---

## 1) 方針（全部 NumPy）

やりたいことはシンプルに言うと：

1. `activity_date` を 30 日間でフィルタ
2. その期間内の `(activity_date, user_id)` ペアをユニークにする
   → 「同じユーザーが同じ日に何回活動しても 1 回」
3. 日別にペア数（= ユニーク user 数）を数える

これを **全部 NumPy の `unique` でやる**、という作戦です。

---

## 2) 実装例（NumPy ゴリ押し版）

LeetCode の Pandas 版シグネチャを想定して、こう書けます：

```python
import pandas as pd
import numpy as np

def daily_active_users(activity: pd.DataFrame) -> pd.DataFrame:
    """
    NumPy メイン実装:
      - フィルタ, 集約はすべて NumPy で実施
      - pandas は列の取り出し & 最終 DataFrame 化のみ
    Returns:
        pd.DataFrame: 列名と順序は ['day', 'active_users']
    """
    # --- 1) 必要列を NumPy 配列として取得 ---
    # 日付は日単位の datetime64[D] にしておくと扱いやすい
    dates = activity["activity_date"].to_numpy(dtype="datetime64[D]")
    users = activity["user_id"].to_numpy()

    # --- 2) 30 日間の期間フィルタ ---
    start = np.datetime64("2019-06-28", "D")
    end = np.datetime64("2019-07-27", "D")
    mask = (dates >= start) & (dates <= end)

    dates = dates[mask]
    users = users[mask]

    if dates.size == 0:
        # 対象期間に何もなければ空 DataFrame を返す
        return pd.DataFrame({"day": pd.Series([], dtype="datetime64[ns]"),
                             "active_users": pd.Series([], dtype="int64")})

    # --- 3) (date, user) ペアを NumPy structured array で表現 ---
    # これで np.unique でペア単位のユニークが取れる
    pairs = np.empty(dates.shape[0], dtype=[("day", "datetime64[D]"), ("user", users.dtype)])
    pairs["day"] = dates
    pairs["user"] = users

    # --- 4) ペアをユニーク化（同じユーザーが同日に複数回いても 1 回に） ---
    uniq_pairs = np.unique(pairs)  # ソートもかかる

    # --- 5) 日付ごとにユニークユーザー数を数える ---
    unique_days, counts = np.unique(uniq_pairs["day"], return_counts=True)

    # --- 6) pandas DataFrame に戻す ---
    # np.datetime64[D] → DataFrame 生成時に datetime64[ns] に昇格する
    result = pd.DataFrame({
        "day": unique_days.astype("datetime64[ns]"),
        "active_users": counts.astype("int64"),
    })

    return result

Analyze Complexity
Runtime 290 ms
Beats 89.97%
Memory 67.06 MB
Beats 99.84%

```

ポイント:

* **グループ化や重複除去は全部 `np.unique`** でやっているので、pandas の `groupby` / `drop_duplicates` は一切使っていません。
* `structured array`（構造化配列）で `(day, user)` のタプルを 1 要素として扱うことで、
  `np.unique` が「行単位でのユニーク」を取ってくれます。
* `np.unique(uniq_pairs["day"], return_counts=True)` で
  「日付ごとの (日, ユーザー) ペア数」をまとめて計算。

---

## 3) アルゴリズムの流れ（NumPy 目線）

1. **カラム抽出（pandas → NumPy）**

   ```python
   dates = activity["activity_date"].to_numpy("datetime64[D]")
   users = activity["user_id"].to_numpy()
   ```

2. **期間フィルタ（ブールマスク）**

   ```python
   mask = (dates >= start) & (dates <= end)
   dates = dates[mask]
   users = users[mask]
   ```

3. **構造化配列で `(day, user)` を表現**

   ```python
   pairs = np.empty(dates.shape[0], dtype=[("day", "datetime64[D]"), ("user", users.dtype)])
   pairs["day"] = dates
   pairs["user"] = users
   ```

4. **ペアのユニーク化**

   ```python
   uniq_pairs = np.unique(pairs)
   ```

   これで「同じ user が同じ day に複数回活動していても一つにまとまる」。

5. **日別にカウント**

   ```python
   unique_days, counts = np.unique(uniq_pairs["day"], return_counts=True)
   ```

   `counts[i]` が `unique_days[i]` のユニーク user 数（= アクティブユーザー数）。

6. **DataFrame へ逆変換**

   ```python
   pd.DataFrame({"day": unique_days.astype("datetime64[ns]"),
                 "active_users": counts.astype("int64")})
   ```

---

## 4) 計算量 & パフォーマンスの雰囲気

`N` = 30 日間に入る行数とすると、

* 期間フィルタ: `O(N)`
* 構造化配列作成: `O(N)`
* `np.unique(pairs)`（ペア単位ユニーク + ソート）:
  `O(N log N)`（内部ソート）くらい
* `np.unique(uniq_pairs["day"], return_counts=True)`:
  ユニークな日数を `D` とすると `O(D log D)`（ただし D ≤ 30 なので誤差）

⇒ **スループットの支配は 2 つの `np.unique`** で、
概ね **O(N log N)** です。
pandas の `groupby().nunique()` と同じオーダーですが、
NumPy 直叩きのぶん、オーバーヘッドはだいぶ小さいです。

---

## 5) おまけ：さらに削るとしたら？

* `activity_date` が最初から `datetime64[ns]` なら、`to_numpy("datetime64[D]")` で日単位に丸める処理はそのまま速いです。
* ユーザー ID を `int32` に落とせる（値の範囲が小さい）のであれば、
  `users.astype("int32")` でメモリを少し削ることもできます。
  （LeetCode だとスコアに出ないことも多いですが。）

---

こんな感じで、ロジックはほぼ NumPy だけで完結させられます。
「pandas 禁止縛りのコードレビュー」みたいな場でも、そのまま説明材料に使えるはずです。